# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import config
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from sklearn.preprocessing import MinMaxScaler
from agents.tfagents.dqn import DQNAgent
from agents.tfagents.ppo import PPOAgent
from environments.environment import TradingEnvironment
from environments.wrappers.tf.tfenv import TFTradingEnvironment
from environments.rewards.marketorder import MarketOrderRF
from environments.rewards.marketlimitorder import MarketLimitOrderRF
from metrics.trading.pnl import CumulativeLogReturn
from metrics.trading.risk import InvestmentRisk
from metrics.trading.sharpe import SharpeRatio
from metrics.trading.sortino import SortinoRatio
from metrics.trading.drawdown import MaximumDrawdown

# Reading Datasets

In [2]:
def read_dataset(
        dataset_filepath,
        timeframe_size,
        target_horizon_len,
        num_eval_samples,
        fees,
        reward_fn_instance
):
    # Reading dataset
    crypto_dataset_df = pd.read_csv(config.dataset_save_filepath.format(dataset_filepath))
    samples_df = crypto_dataset_df[config.regression_features]

    # Scaling data
    scaler = MinMaxScaler(feature_range=(0, 1.0))
    samples = samples_df.to_numpy(dtype=np.float32)

    num_train_scale_samples = samples.shape[0] - num_eval_samples - target_horizon_len - timeframe_size + 1
    samples[: num_train_scale_samples] = scaler.fit_transform(samples[: num_train_scale_samples])
    samples[num_train_scale_samples: ] = scaler.transform(samples[num_train_scale_samples: ])

    # Constructing timeframes for train, test
    inputs = np.float32([samples[i: i + timeframe_size] for i in range(samples.shape[0] - timeframe_size - target_horizon_len + 1)])

    # Splitting inputs to train-test data
    num_train_inputs = inputs.shape[0] - num_eval_samples
    x_train = inputs[: num_train_inputs]
    x_eval = inputs[num_train_inputs:]

    # Computing reward functions for train, test data
    closes = crypto_dataset_df['close'].to_numpy(dtype=np.float32)
    highs = crypto_dataset_df['high'].to_numpy(dtype=np.float32)
    lows = crypto_dataset_df['low'].to_numpy(dtype=np.float32)

    train_reward_fn = reward_fn_instance(
        timeframe_size=timeframe_size,
        target_horizon_len=target_horizon_len,
        highs=highs[: samples.shape[0] - num_eval_samples],
        lows=lows[: samples.shape[0] - num_eval_samples],
        closes=closes[: samples.shape[0] - num_eval_samples],
        fees_percentage=fees
    )

    eval_reward_fn = reward_fn_instance(
        timeframe_size=timeframe_size,
        target_horizon_len=target_horizon_len,
        highs=highs[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        lows=lows[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        closes=closes[samples.shape[0] - num_eval_samples - timeframe_size - target_horizon_len + 1:],
        fees_percentage=fees
    )

    assert x_train.shape[0] == train_reward_fn.get_reward_fn_shape()[0], \
        f'AssertionError: DimensionMismatch: x_train: {x_train.shape}, train_reward_fn: {train_reward_fn.get_reward_fn_shape()}'
    assert x_eval.shape[0] == eval_reward_fn.get_reward_fn_shape()[0], \
        f'AssertionError: DimensionMismatch: x_eval: {x_eval.shape}, eval_reward_fn: {eval_reward_fn.get_reward_fn_shape()}'

    return x_train, train_reward_fn, x_eval, eval_reward_fn

# Building Agent

In [3]:
def build_agent(
        agent_instance,
        observation_spec,
        action_spec,
        time_step_spec,
        env_batch_size,
        checkpoint_filepath,
        fc_layers,
        conv_layers
):
    return agent_instance(
        input_tensor_spec=observation_spec,
        action_spec=action_spec,
        time_step_spec=time_step_spec,
        env_batch_size=env_batch_size,
        checkpoint_filepath=checkpoint_filepath,
        fc_layers=fc_layers,
        conv_layers=conv_layers
    )

# Building Trainer

In [4]:
def train(
        dataset_filepath,
        timeframe_size,
        target_horizon_len,
        num_eval_samples,
        fees,
        reward_fn_instance,
        agent_instance,
        checkpoint_filepath,
        fc_layers,
        conv_layers,
        train_episode_steps,
        train_iterations,
        eval_episodes,
        steps_per_eval,
        steps_per_log,
        steps_per_checkpoint,
        save_best_only
):
    x_train, train_reward_fn, x_eval, eval_reward_fn = read_dataset(
        dataset_filepath=dataset_filepath,
        timeframe_size=timeframe_size,
        target_horizon_len=target_horizon_len,
        num_eval_samples=num_eval_samples,
        fees=fees,
        reward_fn_instance=reward_fn_instance
    )
    train_env = TFTradingEnvironment(
        env=TradingEnvironment(env_config={
            'states': x_train,
            'reward_fn': train_reward_fn,
            'episode_steps': train_episode_steps,
            'metrics': [CumulativeLogReturn(), InvestmentRisk(), SharpeRatio(), SortinoRatio(), MaximumDrawdown()]
        })
    )
    eval_env = TFTradingEnvironment(
        env=TradingEnvironment(env_config={
            'states': x_eval,
            'reward_fn': eval_reward_fn,
            'episode_steps': x_eval.shape[0] - 1,
            'metrics': [CumulativeLogReturn(), InvestmentRisk(), SharpeRatio(), SortinoRatio(), MaximumDrawdown()]
        })
    )

    tf_train_env = TFPyEnvironment(environment=train_env)
    tf_eval_env = TFPyEnvironment(environment=eval_env)

    agent = build_agent(
        agent_instance=agent_instance,
        observation_spec=tf_train_env.observation_spec(),
        action_spec=tf_train_env.action_spec(),
        time_step_spec=tf_train_env.time_step_spec(),
        env_batch_size=tf_train_env.batch_size,
        checkpoint_filepath=checkpoint_filepath,
        fc_layers=fc_layers,
        conv_layers=conv_layers,
    )

    agent.initialize()

    eval_avg_returns = agent.train(
        train_env=tf_train_env,
        eval_env=tf_eval_env,
        train_iterations=train_iterations,
        eval_episodes=eval_episodes,
        iterations_per_eval=steps_per_eval,
        iterations_per_log=steps_per_log,
        iterations_per_checkpoint=steps_per_checkpoint,
        save_best_only=save_best_only
    )
    eval_metrics = eval_env.get_metrics()
    return eval_avg_returns, eval_metrics

# Building Train Configs

In [5]:
datasets_dict = {'BTC': 'BTC', 'ETH': 'ETH', 'ADA': 'ADA', 'XRP': 'XRP', 'LTC': 'LTC'}
rewards_dict = {
    'Market-Orders':  MarketOrderRF,
    'Market-Limit Orders': MarketLimitOrderRF
}
agents_configs = {
    'DDQN': {
        'agent_instance': DQNAgent,
        'train_iterations': 50000,
        'steps_per_eval': 500,
        'steps_per_log': 500,
        'steps_per_checkpoint': 500
    },
    'PPO': {
        'agent_instance': PPOAgent,
        'train_iterations': 1000,
        'steps_per_eval': 10,
        'steps_per_log': 10,
        'steps_per_checkpoint': 10
    }
}
train_dict = {
    'timeframe_size': 12,
    'target_horizon_len': 20,
    'num_eval_samples': 2250,
    'fees': 0.007,
    'fc_layers': [256, 256],
    'conv_layers': [(32, 3, 1)],
    'train_episode_steps': 100,
    'eval_episodes': 1,
    'save_best_only': True
}

# Run TraderNet Experiments

In [6]:
results = {
    'PPO': {dataset_name: {} for dataset_name in datasets_dict.keys()},
    'DDQN': {dataset_name: {} for dataset_name in datasets_dict.keys()}
}

colors = {
    'BTC': 'green',
    'ETH': 'blue',
    'XRP': 'red',
    'ADA': 'black',
    'LTC': 'orange'
}
linestyles = {
    'Market-Orders': '--',
    'Market-Limit-Orders': '-'
}

for agent_name, agent_config in agents_configs.items():
    for dataset_name, dataset_filepath in datasets_dict.items():
        for reward_fn_name, reward_fn_instance in rewards_dict.items():
            tf.random.set_seed(seed=0)

            train_params = {
                'dataset_filepath': dataset_filepath,
                'reward_fn_instance': reward_fn_instance,
                'checkpoint_filepath': f'database/storage/checkpoints/experiments/tradernet/{agent_name}/{dataset_name}/{reward_fn_name}/',
                **train_dict,
                **agent_config
            }
            eval_avg_returns, eval_metrics = train(**train_params)

            results[agent_name][dataset_name][reward_fn_name] = (eval_avg_returns, eval_metrics)

        for reward_fn_name, reward_fn_results in results[agent_name][dataset_name].items():
            eval_avg_returns, eval_metrics = reward_fn_results

            metrics_dict = {
                'steps': [10000*i for i in range(len(eval_avg_returns))],
                'average_returns': eval_avg_returns,
                **{metric.name: metric.episode_metrics for metric in eval_metrics}
            }
            metrics_df = pd.DataFrame(metrics_dict)
            metrics_df.to_csv(f'experiments/tradernet/{agent_name}/{dataset_name}_{reward_fn_name}.csv', index=False)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Training has started...

New best average return found at -3.3251941204071045! Saving checkpoint at iteration 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))



New best average return found at -2.6267342567443848! Saving checkpoint at iteration 500

Iteration: 500
Train Loss: 0.004517136141657829
Average Return: -2.6267342567443848

New best average return found at -2.1988868713378906! Saving checkpoint at iteration 1000

Iteration: 1000
Train Loss: 0.002659676130861044
Average Return: -2.1988868713378906

New best average return found at -1.1928369998931885! Saving checkpoint at iteration 1500

Iteration: 1500
Train Loss: 0.00444418378174305
Average Return: -1.1928369998931885

Iteration: 2000
Train Loss: 0.003950432874262333
Average Return: -1.593271017074585

New best average return found at -0.10673896968364716! Saving checkpoint at iteration 2500

Iteration: 2500
Train Loss: 0.003947971388697624
Average Return: -0.10673896968364716

Iteration: 3000
Train Loss: 0.0027603469789028168
Average Return: -6.610193252563477

Iteration: 3500
Train Loss: 0.003471020143479109
Average Return: -0.64287930727005

New best average return found at 0.26

...


New best average return found at -43.482452392578125! Saving checkpoint at iteration 0

New best average return found at 6.350119590759277! Saving checkpoint at iteration 10

Iteration: 10
Train Loss: 5.717967510223389
Average Return: 6.350119590759277

Iteration: 20
Train Loss: 17.89725685119629
Average Return: -0.9094057083129883

Iteration: 30
Train Loss: 5.808187961578369
Average Return: 2.1024227142333984

Iteration: 40
Train Loss: 12.159335136413574
Average Return: 3.8655874729156494

New best average return found at 8.949485778808594! Saving checkpoint at iteration 50

Iteration: 50
Train Loss: 31.96603012084961
Average Return: 8.949485778808594

New best average return found at 11.190781593322754! Saving checkpoint at iteration 60

Iteration: 60
Train Loss: 87.64916229248047
Average Return: 11.190781593322754

New best average return found at 12.906606674194336! Saving checkpoint at iteration 70

Iteration: 70
Train Loss: 8.28457260131836
Average Return: 12.906606674194336

Ne

C:\Users\kochlis\Documents\Research\TraderNetv2\metrics\trading\sharpe.py:20: RuntimeWarning: invalid value encountered in double_scalars
  return np.exp(average_returns/std_returns)
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\kochlis\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



New best average return found at -58.91674041748047! Saving checkpoint at iteration 0

New best average return found at 26.361770629882812! Saving checkpoint at iteration 10

Iteration: 10
Train Loss: 6.428366184234619
Average Return: 26.361770629882812

Iteration: 20
Train Loss: 40.21513748168945
Average Return: 16.489662170410156

Iteration: 30
Train Loss: 6.854959487915039
Average Return: 15.390291213989258

Iteration: 40
Train Loss: 28.67376136779785
Average Return: 24.604101181030273

Iteration: 50
Train Loss: 27.152767181396484
Average Return: 24.73705291748047

Iteration: 60
Train Loss: 1053.169921875
Average Return: 26.084291458129883

Iteration: 70
Train Loss: 55.041316986083984
Average Return: 20.81643295288086

Iteration: 80
Train Loss: 22.07404136657715
Average Return: 20.322132110595703

Iteration: 90
Train Loss: 9.879684448242188
Average Return: 20.790447235107422

Iteration: 100
Train Loss: 44.56101608276367
Average Return: 19.316007614135742

Iteration: 110
Train Loss:

C:\Users\kochlis\Documents\Research\TraderNetv2\metrics\trading\sortino.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  return np.exp(average_returns/std_downfall_returns)



Iteration: 730
Train Loss: 30.38315773010254
Average Return: 20.944732666015625

Iteration: 740
Train Loss: 26.6074275970459
Average Return: 21.806758880615234

Iteration: 750
Train Loss: 45.98372268676758
Average Return: 25.34726905822754

Iteration: 760
Train Loss: 12.464831352233887
Average Return: 24.05818748474121

Iteration: 770
Train Loss: 23.500608444213867
Average Return: 19.314208984375

Iteration: 780
Train Loss: 14.037674903869629
Average Return: 20.93787956237793

Iteration: 790
Train Loss: 14.23483943939209
Average Return: 24.04638671875

Iteration: 800
Train Loss: 54.20442199707031
Average Return: 24.16851806640625

Iteration: 810
Train Loss: 16.710580825805664
Average Return: 23.73822784423828

Iteration: 820
Train Loss: 39.46760940551758
Average Return: 21.864519119262695

Iteration: 830
Train Loss: 57.25161361694336
Average Return: 25.275304794311523

Iteration: 840
Train Loss: 8.103677749633789
Average Return: 23.893707275390625

Iteration: 850
Train Loss: 28.092546